In [1]:
from Core.GenData import *
from Core.CustomAlgorithm import *
from Core.CustomStructure import *
from Core.APIWarp import *
from Core.Utils import *
from Core.api import *

In [ ]:
target_project  = "D:/work/selector-test/test/quickjs-master"
# target_project  = "D:/tmp/quickjs"
# target_project  = "D:/code/qoi"
target_project  = "D:/tmp/j40"
# target_project  = "D:/tmp/PDFGen"
# target_project  = "D:/work/selector-test/test/PDFGen-master"
target_project  = "D:/work/selector-test/test/j40-main"
target_project  = "D:/work/selector-test/test/olc-master"

In [ ]:
# 初始化CXXProject结构体
test_project = CXXProject(target_project)
test_project.process()

In [ ]:
# 获取备选函数，构造图结构
G = nx.DiGraph()
target_funcs = []
for func in test_project.all_funcs:
    if bytearray2str(func.name) in {"main","LLVMFuzzerTestOneInput"}:
        continue
    target_funcs.append(func)
    # print(func)
G = construct_graph(target_funcs)

In [ ]:
# ParmCode结构体
class ParmCode():
    def __init__(self,one_parm,func_list,model_type):
        self.parm = one_parm
        self.func_list = func_list
        self.parm_code = ""
        self.model_type = model_type
        self.other_parms = []
    
    def is_same_func_list(self,fronzen_set):
        return fronzen_set == frozenset(self.func_list)

    def output_all_parm_name_str(self):
        output_str = f"`{bytearray2str(self.parm.name)}`"
        if self.other_parms:
            output_str += ","
            for one_parm in self.other_parms[:-1]:
                output_str += f"`{bytearray2str(one_parm.name)}`,"
            output_str += f"`{bytearray2str(self.other_parms[-1].name)}`"
        
        return output_str
    def get_func_str(self):
        func_name_str = ""
        for one_func in self.func_list[:-1]:
            func_name_str += f"`{bytearray2str(one_func.name)}`,"
        func_name_str += f"`{bytearray2str(self.func_list[-1].name)}`"
        return func_name_str
    
    def gen_parm_code(self):
        #TODO 当func_list包含函数超过3个时候，需要分批次融合。
        if len(self.func_list) <= 1:
            self.parm_code = self.func_list[0].api_doc
            return

        ins = INS_generate_parm_funcs(self.func_list,self.parm)
        print(f"gen code by parm {self.func_list}")
        logger.info(f"gen code by parm {self.func_list}")
        
        prompt_str = construct_prompt_str(ins,"",self.model_type)
        logger.info(prompt_str)
        self.parm_code = run_llm_custom(prompt_str)
        logger.info(self.parm_code)
        print(self.parm_code)

In [ ]:
def INS_generate_no_parm_funcs(func_list):
    func_name_str = ""
    func_api_doc = ""
    for i,one_func in enumerate(func_list[:-1]):
        func_name_str += f"`{bytearray2str(one_func.name)}`,"
        func_api_doc += f"## 函数{i+1} {bytearray2str(one_func.name)}\n"
        func_api_doc += one_func.api_doc + "\n\n"
        
    func_name_str += f"`{bytearray2str(func_list[-1].name)}`"
    func_api_doc += f"## 函数{len(func_list)} {bytearray2str(func_list[-1].name)}\n"
    func_api_doc += func_list[-1].api_doc
    
    ins = f"""
下面列出了{len(func_list)}个函数的功能和调用片段，根据函数使用场景，写出符合{len(func_list)}个函数{func_name_str}调用场景的同时调用{len(func_list)}个函数的示例代码。
回答要尽可能精准和简洁，代码要尽可能的优化且要符合函数功能的逻辑调用关系。
注意：必须确保每个函数{func_name_str}都必须被显示的调用，最后要解释{func_name_str}都是如何被调用的。
生成完毕后，检查生成的代码，检查{func_name_str}函数是否都被显示的调用了，如果没有，则重新生成代码。
注意：生成的代码均是C语言代码，而不是C++的。

{func_api_doc}
"""
    return ins  

In [ ]:
# 负责生成C工程harness的类
class CGenerator1():
    def __init__(self,selector,project,model_type="codeqwen"):
        self.selector = selector
        self.project = project
        self.model_type = model_type
    def filter_func(self):
        self.selector.filter_func()
    #对具有Process结构体的函数生成函数api文档（文档形式暂时未定），不对init函数和destory函数生成，这两种函数只需要知道函数定义即可调用。
    # TODO 定义文档基本格式和要素内容    
    def gen_api_doc(self):
        # init 获取api doc
        # init 函数只需要给出函数摘要就可以
        for one_func in self.selector.init_funcs:
            # print(f"gen init api doc {one_func.name}")
            # logger.info(f"gen init api doc {one_func.name}")
            # prompt_str = construct_prompt_str(INS_generate_api_doc(one_func,self.selector),bytearray2str(one_func.body),self.model_type)
            # logger.info(prompt_str)
            # api_doc = run_llm_custom(prompt_str)
            api_doc = bytearray2str(one_func.body).split("{")[0].replace("\n","")
            # print(api_doc)
            # logger.info(api_doc)
            one_func.api_doc = api_doc
        # destory 函数获取api doc
        for one_func in self.selector.destory_funcs:
            func_head = bytearray2str(one_func.body).split('{')[0].replace("\n","")
            one_func.api_doc = func_head
        # input 获取api doc
        print(f"gen input func api doc {self.selector.input_func.name}")
        logger.info(f"gen input api doc {self.selector.input_func.name}")
        prompt_str = construct_prompt_str(INS_generate_api_doc(self.selector.input_func,self.selector),bytearray2str(self.selector.input_func.body),self.model_type)
        logger.info(prompt_str)
        api_doc = run_llm_custom(prompt_str)
        self.selector.input_func.api_doc = api_doc
        logger.info(api_doc)
        # process函数获取api doc
        for one_func in self.selector.process_funcs:
            print(f"gen process api doc {one_func.name}")
            logger.info(f"gen process api doc {one_func.name}")
            prompt_str = construct_prompt_str(INS_generate_api_doc(one_func,self.selector),bytearray2str(one_func.body),self.model_type)
            logger.info(prompt_str)
            api_doc = run_llm_custom(prompt_str)
            logger.info(api_doc)
            one_func.api_doc = api_doc

    def gen_code_by_parm(self):
        # 根据对输入参数这个维度将函数分为若干组
        #TODO 当func_list包含函数超过3个时候，需要分批次融合。
        all_input_parms = self.selector.get_input_parms()
        self.parm_code_list = []
        self.parm_func_name_list = []
        self.input_parm_code_dict = {}
        analyzed_funcs_set = set()
        if len(all_input_parms) == 0:
            # 如果没有input parm，说明整个Input funcs就没有处理结构体，那么大概率就是单独函数，直接生成robo function
            all_funcs = self.selector.get_all_funcs()
            prompt_str = INS_generate_no_parm_funcs(all_funcs)
            self.robo_code = run_llm_custom(prompt_str)
            return 
        # 暂时存储在这里
        for one_parm in all_input_parms:
            func_list = self.selector.get_input_parm_funcs(one_parm.name)
            funcs_set = frozenset(func_list)
            # 防止出现重复分析相同的set
            if funcs_set in analyzed_funcs_set:
                for key in self.input_parm_code_dict:
                    if self.input_parm_code_dict[key].is_same_func_list(funcs_set):
                        self.input_parm_code_dict[key].other_parms.append(one_parm)
                continue
            analyzed_funcs_set.add(funcs_set)
            one_parm_code = ParmCode(one_parm,func_list,self.model_type)
            one_parm_code.gen_parm_code()
            self.input_parm_code_dict[one_parm] = one_parm_code
            
            # self.parm_func_name_list.append(func_name_str)
            
            
        
    def gen_robo_code(self):
        # 如果没有任何输入参数，那么说明只有单独一个input func，那么就直接跳过流程，直接生成harness code
        if not self.input_parm_code_dict:
            return 
        # 用有向图建立不同函数输入参数和输出参数之间的关系，形成参数的调用流程。进而对参数进行排序。
        parm_graph = nx.DiGraph()
        parm_list_sorted = []
        parm_dict = {}
        input_parms = self.selector.get_input_parms()
        for one_parm in input_parms:
            parm_graph.add_node(one_parm)
        all_funcs = self.selector.get_all_funcs()
        # 寻找处理输入结构体并返回结构体的函数，这样能够展示出结构体的流动状态，那些不在图里的结构体，暂定优先级是最高的。
        for one_func in all_funcs:
            if one_func.input_parm_set and one_func.output_parm_set:
                for out_parm in one_func.output_parm_set:
                    for in_parm in one_func.input_parm_set:
                        parm_graph.add_edge(in_parm,out_parm)
        for one_parm in self.input_parm_code_dict:
            parm_dict[one_parm] = len(list(nx.ancestors(parm_graph,one_parm)))
        sorted_keys = sorted(parm_dict, key=lambda x: parm_dict[x])
        for key in sorted_keys:
            parm_list_sorted.append(key)
        # 排序后的parm_list在parm_list_sorted中。

        # 根据结构体的情况，采用两两结合，按照逻辑顺序的方式生成代码。
        first_parm = parm_list_sorted[0]
        first_parm_code = self.input_parm_code_dict[first_parm]
        robo_dict = {}
        # robo_dict['parm_set'] = set(first_parm_code.parm) | set(first_parm_code.other_parms)
        robo_dict['parm_set'] = set([first_parm_code.parm] + list(first_parm_code.other_parms))
        robo_dict['func_list'] = first_parm_code.func_list
        robo_dict['code'] = first_parm_code.parm_code
        if len(parm_list_sorted) <= 1:
            self.robo_code = robo_dict["code"]
            return
        
        for one_parm in parm_list_sorted[1:]:
            if one_parm in robo_dict['parm_set']:
                continue
            parm_code = self.input_parm_code_dict[one_parm]
            print(f"gen robo code {robo_dict['parm_set']} {parm_code.parm}")
            logger.info(f"gen robo code {robo_dict['parm_set']} {parm_code.parm}")
            
            ins = INS_generate_robo_code(robo_dict,parm_code)
            prompt_str = construct_prompt_str(ins,"",self.model_type)
            logger.info(prompt_str)
            new_robo_code = run_llm_custom(prompt_str)
            logger.info(new_robo_code)
            robo_dict["parm_set"] |= set([parm_code.parm] + list(parm_code.other_parms))
            robo_dict['func_list'] += parm_code.func_list
            robo_dict['code'] = new_robo_code
            
        self.robo_code = robo_dict["code"]
            
    def gen_opt_code(self):
        # 如果robo code为空，说明不需要这个步骤（单独函数存在时候）
        if not self.robo_code:
            prompt_str = construct_prompt_str(INS_generate_opt_code(),self.opt_code,"codeqwen")
            logger.info(f"gen opt code\n{prompt_str}")
            self.opt_code = run_llm_custom(prompt_str)
            logger.info(self.opt_code)
            return 
        prompt_str = construct_prompt_str(INS_generate_opt_code(),self.robo_code,"codeqwen")
        print("gen opt code")
        logger.info(f"gen opt code\n{prompt_str}")
        self.opt_code = run_llm_custom(prompt_str)
        logger.info(self.opt_code)
    def gen_harness_code(self):
        prompt_str = construct_prompt_str(INS_generate_harness_code(),self.opt_code,"codeqwen")
        print(prompt_str)
        print("gen harness code")
        logger.info(f"gen harness code\n{prompt_str}")
        self.harness_code = run_llm_custom(prompt_str)
        logger.info(self.harness_code)

In [ ]:
from Gen import *
all_func_list = []
for one_func in target_funcs:
    if one_func.name == b'j40_from_memory':
        print(one_func)
        test_g = OneSelectorGroup(one_func)
        all_func_list.append(one_func)
#b'j40_from_memory' process funcs [j40_current_frame:[], j40__check_image:[], j40_error:[], j40_frame_pixels_u8x4:[], j40_error_string:[], j40_next_frame:[], j40_output_format:[]]
for one_func in target_funcs:
    if one_func.name == b'j40_next_frame':
        test_g.process_funcs.append(one_func)
        all_func_list.append(one_func)
    elif one_func.name == b'j40_current_frame':
        test_g.process_funcs.append(one_func)
        all_func_list.append(one_func)
    elif one_func.name == b'j40_frame_pixels_u8x4':
        test_g.process_funcs.append(one_func)
        all_func_list.append(one_func)
    elif one_func.name == b'j40_output_format':
        test_g.process_funcs.append(one_func)
        all_func_list.append(one_func)
    elif one_func.name == b'j40__check_image':
        test_g.process_funcs.append(one_func)
        all_func_list.append(one_func)
    elif one_func.name == b'j40_error':
        test_g.process_funcs.append(one_func)
        all_func_list.append(one_func)
    elif one_func.name == b'j40_error_string':
        test_g.process_funcs.append(one_func)
        all_func_list.append(one_func)
    elif one_func.name == b'j40_free':
        test_g.destory_funcs.append(one_func)
        all_func_list.append(one_func)

# Selector = CSelector(target_funcs,test_project,"codeqwen",1)
Selector = CSelector(all_func_list,test_project,"codeqwen",1)
Selector.analyze_funcs_parm(all_func_list)


In [ ]:
from Gen import *
candi_list = [] 
for one_func in test_project.all_funcs:
    if bytearray2str(one_func.name) in {"main","LLVMFuzzerTestOneInput"}:
        continue
    candi_list.append(one_func)
Selector = CSelector(candi_list,test_project,"codeqwen",1)
selector_list = Selector.run()

In [ ]:
for st in selector_list:
    print(st.show())
    generator = CGenerator1(st,test_project)
    generator.filter_func()
    generator.gen_api_doc()
    generator.gen_code_by_parm()
    generator.gen_robo_code()
    generator.gen_opt_code()
    generator.gen_harness_code()
    print(generator.harness_code)
    break
 

In [ ]:
# 测试API doc生成
generator.gen_api_doc()

In [ ]:
generator.gen_code_by_parm()

In [ ]:
print(generator.input_parm_code_dict)
generator.gen_robo_code()

In [ ]:
print(generator.robo_code)
generator.gen_opt_code()

In [ ]:
generator.gen_harness_code()
print(generator.harness_code)

In [3]:
# 测试re generate 
import subprocess
from Core.promptINS import *
old_fuzzer_code = """
#define J40_IMPLEMENTATION
#define J40_CONFIRM_THAT_THIS_IS_EXPERIMENTAL_AND_POTENTIALLY_UNSAFE
#include "j40.h"

#include <stdlib.h>

void display_pixels(j40_pixels_u8x4 pixels) {
    (void)pixels; // Suppress unused variable warning.
}

int LLVMFuzzerTestOneInput(const uint8_t *data, size_t data_size) {
    j40_image img;

    if (j40_from_memory(&img, (void *)data, data_size, free) != 0)
        return 1;

    if (j40_error(&img) != 0) {
        j40_free(&img);
        return 1;
    }

    if (j40_output_format(&img, 1, 4) != 0) {
        j40_free(&img);
        return 1;
    }

    while (j40_next_frame(&img)) {
        j40_frame currentFrame = j40_current_frame(&img);

        if (currentFrame.magic == 0x12345678) {
            j40_pixels_u8x4 pixels = j40_frame_pixels_u8x4(&currentFrame, 1);
            if (pixels.data != NULL)
                display_pixels(pixels); 
        } else {
            break;
        }
    }

    // Free resources.
    j40_free(&img);

    return 0; // Fuzzer requires int as the return type
}"""
fuzzer_code = """
#define J40_CONFIRM_THAT_THIS_IS_EXPERIMENTAL_AND_POTENTIALLY_UNSAFE
#define J40_IMPLEMENTATION
#include "j40.h"

// �Զ�����ڴ��ͷź���ʾ������Ȼ�� Fuzz �����в�ʹ�ã���Ϊ�������Ա�����
void custom_free_func(void* buf) { free((char*)buf); }

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
    // ����������ݵĴ�С�Ƿ�Ϊ0�����Ϊ0��ֱ�ӷ���
    if (Size == 0)
        return 0;

    j40_image img;
    
    // ʹ��j40_from_memory������ʼ��������ͼ������
    j40_err err = j40_from_memory(&img, (void*)Data, Size, custom_free_func);
    if(err) {
        return 0;  // �� Fuzz �����У������ش�������Ƿ��� 0 ��ʾʧ��
    }

    // ʹ��j40_output_format��������ͼ�������ʽ
    err = j40_output_format(&img, J40_RGBA, J40_U8X4);
    if(err) {
        return 0;
    }
    
    // ѭ����ȡ������ÿһ֡
    while (j40_next_frame(&img)) {  
        j40_frame frame = j40_current_frame(&img);

        // ʹ��j40_frame_pixels_u8x4������ȡ֡����������
        if(frame.magic == J40__FRAME_MAGIC) {
            j40_pixels_u8x4 rgbaPixels = j40_frame_pixels_u8x4(&frame, J40_RGBA);
            
            if (rgbaPixels.data != NULL) {  // ʹ����ȷ�Ĵ����鷽��    
                uint32_t* firstPixel = (uint32_t*)rgbaPixels.data;
                
                // ��������Խ���һЩ���������ݵļ������
                // ���磬�򵥵Ķ�ȡ��һ�������Դ���Ǳ�ڵ�����
                (void)*firstPixel;  // ʹ�� volatile ����ֹ�������Ż����������
                
            } 
        }
    }

    return 0;
}

"""
gen_file_path="D:/work/selector-test/test/j40-main/test.c"
target_floder="D:/work/selector-test/test/j40-main"
gen_command="wsl -d Ubuntu-24.04 -e bash -c \"cd /mnt/d/work/selector-test/test/j40-main && clang -fsanitize=fuzzer,address -fprofile-instr-generate -fcoverage-mapping -g -O2 test.c\""
run_command="wsl -d Ubuntu-24.04 -e bash -c \"cd /mnt/d/work/selector-test/test/j40-main && ./a.out -max_total_time=10 -detect_leaks=0\""

with open(gen_file_path, 'w') as f:
    f.write(fuzzer_code)


process = subprocess.Popen(gen_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
stdout, error_message = process.communicate()
print(error_message)
process = subprocess.Popen(run_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
stdout, error_message = process.communicate()
print(stdout)
print("<====================================================>")
print(error_message)
if len(error_message.split("\n")) > 100:
    print("good example")
    return 
ins = INS_regenerate_run(fuzzer_code,error_message)
prompt_str = construct_prompt_str(ins,"","codeqwen")
for _ in range(3):
    result = run_llm_custom(prompt_str)
    print(result)
    content_list = result.split("```c")
    fuzzer_code = ""
    for one_code in content_list[::-1]:
        if "LLVMFuzzerTestOneInput(" in one_code:
            fuzzer_code = one_code.split("```")[0]
            break
    with open(gen_file_path, 'w') as f:
        f.write(fuzzer_code)
    process = subprocess.Popen(gen_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
    stdout, error_message = process.communicate()
    print("compile:",error_message)
    print("compile len:",len(error_message))
    if len(error_message) > 10:
        print("compile error")
    else:
        process = subprocess.Popen(run_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
        stdout, error_message = process.communicate()
        # print("run:",stdout)
        print("<====================================================>")
        print("run:",error_message)
        break



<====================================================>
INFO: Running with entropic power schedule (0xFF, 100).
INFO: Seed: 1581986355
INFO: Loaded 1 modules   (4934 inline 8-bit counters): 4934 [0x55d274203c10, 0x55d274204f56), 
INFO: Loaded 1 PC tables (4934 PCs): 4934 [0x55d274204f58,0x55d2742183b8), 
INFO: -max_len is not provided; libFuzzer will not generate inputs larger than 4096 bytes
INFO: A corpus is not provided, starting from an empty corpus
#2	INITED cov: 32 ft: 33 corp: 1/1b exec/s: 0 rss: 40Mb
	NEW_FUNC[1/1]: 0x55d2740f1120 in j40__memory_source_read /mnt/d/work/selector-test/test/j40-main/./j40.h:1220
#4	NEW    cov: 38 ft: 39 corp: 2/3b lim: 4 exec/s: 0 rss: 41Mb L: 2/2 MS: 2 ChangeBit-CrossOver-
#33	NEW    cov: 39 ft: 40 corp: 3/5b lim: 4 exec/s: 0 rss: 42Mb L: 2/2 MS: 4 CopyPart-ChangeByte-EraseBytes-InsertByte-
#128	NEW    cov: 40 ft: 44 corp: 4/8b lim: 4 exec/s: 0 rss: 44Mb L: 3/3 MS: 5 ShuffleBytes-ChangeByte-CrossOver-EraseBytes-CopyPart-
#216	NEW    cov: 41 ft: 

SyntaxError: 'return' outside function (2427583298.py, line 117)